# Getting Embeddings

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import pandas as pd
import torch.nn as nn
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
# from matplotlib import pyplot as plt

# The model is running on CPU, since it is already pre-trained and doesnt require GPU
device = torch.device('cpu') 
print('Running on device: {}'.format(device))

#Define MTCNN module
#Since MTCNN is a collection of neural nets and other code, 
#The device must be passed in the following way to enable copying of objects when needed internally.
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device
)
#Function takes 2 vectors 'a' and 'b'
#Returns the cosine similarity according to the definition of the dot product
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

#cos_sim returns real numbers,where negative numbers have different interpretations.
#So we use this function to return only positive values.
def cos(a,b):
    minx = -1 
    maxx = 1
    return (cos_sim(a,b)- minx)/(maxx-minx)

# Define Inception Resnet V1 module (GoogLe Net)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define a dataset and data loader
dataset = datasets.ImageFolder('frames')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=lambda x: x[0])

#Perfom MTCNN facial detection
#Detects the face present in the image and prints the probablity of face detected in the image.
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

# Calculate the 512 face embeddings
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).cpu()

# Print distance matrix for classes.
#The embeddings are plotted in space and cosine distace is measured.
cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-6)
for i in range(0,len(names)):
    emb=embeddings[i].unsqueeze(0)
    # The cosine similarity between the embeddings is given by 'dist'.
    dist =cos(embeddings[0],emb)  
        
dists = [[cos(e1,e2).item() for e2 in embeddings] for e1 in embeddings]
# The print statement below is
#Helpful for analysing the results and for determining the value of threshold.
print(pd.DataFrame(dists, columns=names, index=names)) 


Running on device: cpu
Face detected with probability: 0.999992
Face detected with probability: 0.999946
Face detected with probability: 0.999997
Face detected with probability: 1.000000
Face detected with probability: 1.000000
Face detected with probability: 1.000000
Face detected with probability: 0.999998
Face detected with probability: 0.999918
Face detected with probability: 0.998948
Face detected with probability: 0.996932
Face detected with probability: 0.999997
Face detected with probability: 1.000000
Face detected with probability: 0.999996
Face detected with probability: 0.999996
Face detected with probability: 1.000000
Face detected with probability: 0.999994
Face detected with probability: 1.000000
Face detected with probability: 0.999974
Face detected with probability: 0.999980
Face detected with probability: 0.999998
Face detected with probability: 0.999977
Face detected with probability: 0.999997
Face detected with probability: 0.999986
Face detected with probability: 0.

# Background Cycle

In [2]:
def background_subtraction(previous_frame, frame_resized_grayscale, min_area):
    """
    This function returns 1 for the frames in which the area
    after subtraction with previous frame is greater than minimum area
    defined.
    Thus expensive computation of human detection face detection
    and face recognition is not done on all the frames.
    Only the frames undergoing significant amount of change (which is controlled min_area)
    are processed for detection and recognition.
    """
    frameDelta = cv2.absdiff(previous_frame, frame_resized_grayscale)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
    # cv2.imshow("Thresh",thresh)
    # cv2.waitKey(200)
    thresh = cv2.dilate(thresh, None, iterations=2)
    # cv2.imshow("Thresh dialtetd",thresh)
    # cv2.waitKey(200)
    countours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    temp = 0
    for c in countours:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) > min_area:
            temp = 1
    return temp

# Face Recognition from Images

In [3]:
import glob
import os
import sys
import time
from facenet_pytorch import MTCNN, InceptionResnetV1,extract_face
from PIL import Image,ImageDraw
import torch
import cv2
import imutils
import torch.nn as nn

target_name = 'avi'
# result output video
output_video_name = f"avi-neu.avi"
# for writing frame
output_fps = 23
# for frame dropping
dropped = 0
is_detected = False


#Takes 2 vectors 'a' and 'b'.
#Return the cosine similarity according to the definition of the dot product.
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

#cos_sim returns real numbers,where negative numbers have different interpretations.
#So we use this function to return only positive values.
def cos(a,b):   
    minx = -1 
    maxx = 1
    return (cos_sim(a,b)- minx)/(maxx-minx)


#This is the function for doing face recognition.
def verify(embedding,writer,not_detected,is_detected): 
    for j,l in enumerate(embedding):
        minDist = float("-inf")
        ansi, ansj = 0, 0
        for i,k in enumerate(embeddings):
            #Computing Cosine distance.
            dist =cos(k,l)
            if(dist>minDist):
                minDist = dist
                ansi = i
                ansj = j                
            # Chosen threshold is 0.85. 
            #Threshold is determined after seeing the table in the previous cell.
        #Name of the person identified is printed on the screen, as well as below the detecetd face (below the rectangular box).
        if minDist > 0.80:
            text=names[ansi]
            cv2.putText(im, text,(boxes[ansj][0].astype(int) ,boxes[ansj][3].astype(int) + 17), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 2)
            if text==target_name: 
                is_detected = True
                writer.write(im)
                not_detected = 0

            else:
                not_detected += 1
                if not_detected>=30:
                    is_detected = False
                    not_detected = 0
                elif is_detected:
                    writer.write(im)
            print(text)

    return is_detected,not_detected
                
#Model running on CPU           
device = torch.device('cpu')

#Define Inception Resnet V1 module (GoogLe Net)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to('cpu')

#Define MTCNN module
#Since MTCNN is a collection of neural nets and other code, 
#The device must be passed in the following way to enable copying of objects when needed internally.
#'keep_all' is kept True. All the faces present in the image will be detected.
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device,keep_all=True
)
# camera = cv2.VideoCapture("/home/divyanshs/Documents/arpit major final/FaceNet_FR-master/Test Videos/avi ka video.mp4")
# #Get cropped and prewhitened image tensor of PIL image.
# i = 0
# while True:
#     ret, img = camera.read()
path = "avi"
for f in os.listdir(path):
    list_of_videos = glob.glob(os.path.join(os.path.abspath(path), f))
    print(os.path.join(os.path.abspath(path), f))
    print(list_of_videos)
    for video in list_of_videos:
            total_frame = 0
            frame_dropped = 0
            face_detected = 0
            camera = cv2.VideoCapture(os.path.join(path, video))
            grabbed, frame = camera.read()
            # print(frame.shape)
            # width 800 HYPERPARAMETER
            frame_resized = imutils.resize(frame, width=min(800,frame.shape[1]))
            frame_resized_grayscale = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
            height, width, layers = frame_resized.shape
            # Decaaring video writer
            writer = cv2.VideoWriter(output_video_name,cv2.VideoWriter_fourcc(*'MPEG'),output_fps,(width,height))

            if not writer:
                print("Error in creating video writer")
                sys.exit(1)
            

            # print(frame_resized.shape)
            # HYPERPARAMETER
            # defining min cuoff area
            min_area = (3000 / 800) * frame_resized.shape[1]
            i= 0
            starttime = time.time()
            not_detected = 0
            while True:
                previous_frame = frame_resized_grayscale
                grabbed, frame = camera.read()
                # Printing frame for debugging purpose
                if not grabbed: break
                # cv2.imshow("Frame",previous_frame)
                # cv2.waitKey(1)
                # end
                frame_resized = imutils.resize(frame, width=min(800, frame.shape[1]))
                frame_resized_grayscale = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
                # i+=1
                # if(i%10!=0): continue
                
                temp = background_subtraction(previous_frame, frame_resized_grayscale, min_area)
                total_frame += 1
                img = frame_resized
                
                if temp == 1:
                    print("loop")
                    # if img.shape[1]<img.shape[0]:
                    # img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                    # img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

                    img = imutils.resize(img, width=min(800, img.shape[1]))
                    cv2.imwrite("frame.jpg", img)
                    cv2.imshow("Original",img)
                    
                    cv2.waitKey(1)
                    # i += 1
                    # if i%10: continue
                    img = Image.open("frame.jpg")
                    img_cropped = mtcnn(img)
                    boxes,prob=mtcnn.detect(img)
                    img_draw = img.copy()
                    draw = ImageDraw.Draw(img_draw)

                    #Rectangular boxes are drawn on faces present in the image.
                    #The detected and cropped faces are then saved.
                    if boxes is not None and len(boxes)>=1:
                        for i, box in enumerate(boxes):
                            draw.rectangle(box.tolist())
                            extract_face(img, box, save_path='Film/Test/Cropped_Face_{}.jpg'.format(i))
                        img_draw.save('Film/Test/Faces_Detected.jpg')

                        #Calculate embeddings of each cropped face and print it.
                        im=cv2.imread('Film/Test/Faces_Detected.jpg')
                        img_embedding = resnet(img_cropped)
                        # print(img_embedding)

                        #print(size of img_embedding)
                        print(img_embedding.size())

                        #Call function verify. 
                        #Identify the person with the help of embeddings.
                        cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-6)
                        is_detected,not_detected = verify(img_embedding,writer,not_detected,is_detected)

                        #'Image' window opens.
                        #The PIL image now have rectangular boxes on detected faces.
                        #The identified faces have their respective name below the box.
                        cv2.imshow("Image",im)
                        cv2.waitKey(1)
                
                else:
                    print("Dropped frame")
                    # if is_detected:writer.write(img);dropped+=1
            print(f"Numbers of frames dropped and taken {dropped}")
            writer.release()

    
# cv2.waitKey(0)

#13 is for 'Enter' key.
#If 'Enter' key is pressed, all the windows are made to close forcefully.
# if k==13:
cv2.destroyAllWindows()
    

/home/divyanshs/Documents/arpit major final/FaceNet_FR-master/avi/avi ka video.mp4
['/home/divyanshs/Documents/arpit major final/FaceNet_FR-master/avi/avi ka video.mp4']
loop
loop
loop
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
loop
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
loop
Dropped frame
loop
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dropped frame
Dr